In [1]:
import numpy as np
import xarray as xr
from distributed import Client
import dask_jobqueue
import dask
#import daskdataframe as dd
import matplotlib.pyplot as plt
import glob
import netCDF4 as nc
import zarr
import nc_time_axis
import xskillscore as xs
import cartopy as cart
#%run data_handler.ipynb
#np.set_printoptions(threshold=np.inf)

In [10]:
test = xr.DataArray(np.arange(1,101).reshape(50,2), coords=[np.arange(50), np.arange(2)], dims=["time", "member"])

In [13]:
test_stacked = test.stack(tm=["time", "member"])

In [36]:
N_members = 2
window_len = 3
test_construct = test_stacked.rolling(tm=N_members*window_len, center=True).construct('index', stride=N_members)

In [37]:
test_construct.quantile(.5, dim='index')

<xarray.DataArray (tm: 50)>
array([ 2. ,  3. ,  4.5,  6.5,  8.5, 10.5, 12.5, 14.5, 16.5, 18.5, 20.5,
       22.5, 24.5, 26.5, 28.5, 30.5, 32.5, 34.5, 36.5, 38.5, 40.5, 42.5,
       44.5, 46.5, 48.5, 50.5, 52.5, 54.5, 56.5, 58.5, 60.5, 62.5, 64.5,
       66.5, 68.5, 70.5, 72.5, 74.5, 76.5, 78.5, 80.5, 82.5, 84.5, 86.5,
       88.5, 90.5, 92.5, 94.5, 96.5, 98. ])
Coordinates:
  * tm        (tm) object MultiIndex
  * time      (tm) int64 0 1 2 3 4 5 6 7 8 9 ... 40 41 42 43 44 45 46 47 48 49
  * member    (tm) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0 0
    quantile  float64 0.5

In [4]:
test_roll = test.rolling(x=3, center=True)

In [5]:
test_window = test_roll.construct("index")

In [6]:
test_window.sel(index=2)

<xarray.DataArray (x: 10)>
array([ 2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., nan])
Coordinates:
  * x        (x) int64 20 21 22 23 24 25 26 27 28 29

In [8]:
test_window.sel(index=0)

<xarray.DataArray (x: 10)>
array([nan,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.])
Coordinates:
  * x        (x) int64 20 21 22 23 24 25 26 27 28 29

In [39]:
test_window.sel(x=6)

<xarray.DataArray (window: 5)>
array([2., 3., 4., 5., 6.])
Coordinates:
    x        int64 6
Dimensions without coordinates: window

In [ ]:
#Test if the rolling construct and quantiles are working properly on arr
arr = xr.DataArray(np.arange(0, 9, 0.5).reshape(3, 6), coords=[np.arange(0,3), np.arange(5,11)], dims=["x","y"])
arr

In [10]:
arr_roll = arr.rolling(y=2,center=True,min_periods=2)

In [12]:
arr_window = arr_roll.construct("window")

In [18]:
arr_window.isel(window=0)

<xarray.DataArray (x: 3, y: 6)>
array([[nan, 0. , 0.5, 1. , 1.5, 2. ],
       [nan, 3. , 3.5, 4. , 4.5, 5. ],
       [nan, 6. , 6.5, 7. , 7.5, 8. ]])
Coordinates:
  * x        (x) int64 0 1 2
  * y        (y) int64 5 6 7 8 9 10

In [16]:
arr_window.stack(xy=["x","y"]).isel(window=0)

<xarray.DataArray (xy: 18)>
array([nan, 0. , 0.5, 1. , 1.5, 2. , nan, 3. , 3.5, 4. , 4.5, 5. , nan,
       6. , 6.5, 7. , 7.5, 8. ])
Coordinates:
  * xy       (xy) object MultiIndex
  * x        (xy) int64 0 0 0 0 0 0 1 1 1 1 1 1 2 2 2 2 2 2
  * y        (xy) int64 5 6 7 8 9 10 5 6 7 8 9 10 5 6 7 8 9 10